In [18]:
# from dotenv import load_dotenv
import os

from google.colab import userdata
# HF_TOKEN = userdata.get('HF_TOKEN')
OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')

# load_dotenv()
# huggingface_token = os.getenv("HUGGINGFACE_TOKEN")
# OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [3]:
from google.colab import drive
import os
import zipfile
from tqdm import tqdm
# from dotenv import load_dotenv
from openai import OpenAI
# from transformers import pipeline, AutoTokenizer, AutoModelForSeq2SeqLM

In [11]:
# ========== Mount Google Drive ==========
drive.mount('/content/drive')

# ========== Path Configuration ==========
# Update these paths according to your Google Drive structure
DRIVE_BASE = '/content/drive/MyDrive/'
ZIP_PATH = "./data/elmundo_chunked_es_page1_40years.zip"
EXTRACT_DIR = '/tmp/extracted'  # Using tmp for faster I/O
OUTPUT_DIR = os.path.join(DRIVE_BASE, 'cleaned_articles1')

# Create directories
os.makedirs(EXTRACT_DIR, exist_ok=True)
os.makedirs(OUTPUT_DIR, exist_ok=True)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
ZIP_PATH = "./data/elmundo_chunked_es_page1_40years.zip"
EXTRACT_DIR = '/tmp/extracted'  # Using tmp for faster I/O
os.makedirs(EXTRACT_DIR, exist_ok=True)

In [6]:
# ========== File Extraction ==========
def extract_files():
    with zipfile.ZipFile(ZIP_PATH, 'r') as zip_ref:
        # Extract nested structure
        for file in zip_ref.namelist():
            if file.endswith('.txt'):
                zip_ref.extract(file, EXTRACT_DIR)
    print("*" * 50)
    print(f"Extracted files to: {EXTRACT_DIR}")

1. Extract zip file
2. open the folder
3. For each file in folder,
    read the content and extract the text
    <!-- chunk the text into 1000 words -->
    <!-- pass chunks to the model so it can fix the spelling -->
    translate the corrected text to english
    <!-- add the file name and the corrected text to a dictionary -->
    save the corrected text to a new file
4. Save the dictionary to a pkl file
5.


In [19]:
from openai import OpenAI
client = OpenAI(
    api_key=OPENAI_API_KEY
)

def correct_with_openai(text, filename, just_text = True, max_completion_tokens = 2048, temperature = 1, top_p = 1, frequency_penalty=0, presence_penalty=0,**kwargs):
  response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
      {
        "role": "user",
        "content": [
          {
            "type": "text",
            "text": f"Eres un experto en documentos históricos de Puerto Rico. El texto en español son noticias del siglo XX y contiene muchos errores a causa del OCR. Descifra el contenido y tradúcelo al inglés:\n1. Preserva nombres propios (ej: Mayagüez, Caguas)\n2. Ignora el \"header\" (ej:\n```EL MUNDO\nPRONOSTICOS DEL TIEMPO PARA LA ISLA, HOY: Mayormente nublado, con aguaceros dispersos temprano en la mafiana. EN SAN JUAN. AYER: Temperatura máxima. 80; mínima, 77. Presión barométrica al nivel del mar, a las 4:80 de la tarde. 38.88 pulgadas de mercurio. No hay indicios de disturbio tropical.\n40 páginas 5/\nDIARIO DE LA MARANA\nAÑO XXVIII\nEntered aa second clsss matter, Post Office, San Juan, P. R.)```\n3. Ignora los anuncios\n4. Solo mantén contenido relacionado a Puerto Rico (especialmente sobre ciudades, locaciones o eventos históricos)\n5. Traduce el texto a inglés. Solo mantén los datos mas importantes\n6.  Lista las ciudades o locaciones de Puerto Rico mencionadas\n7. Escribe solo en texto (no uses **negrillas** ni *itálicas* ni nada en markdown)\n8. return it as a JSON object with two fields:\n    - 'metadata': un diccionario con la siguiente informacion: 'filename' (nombre del articulo), 'date' (fecha del articulo), 'locations' (lista de las ciudades o locaciones de Puerto Rico mencionadas).\n    - 'text': the corrected and summarized text in English.\n8. No digas nada mas ni preguntes más. El nombre del articulo es {filename}. Usa el siguiente texto: {text}"
          }
        ]
      }
    ],
    response_format={
      "type": "json_object"
    },
    temperature=temperature,
    max_completion_tokens=max_completion_tokens,
    top_p=top_p,
    frequency_penalty=frequency_penalty,
    presence_penalty=presence_penalty,
    **kwargs
  )
  if just_text:
    return response.choices[0].message.content

  return response

In [8]:
from datetime import datetime
import pickle as pkl

def save_progress(data, filename="all_docs.pkl"):
    """ Save the current state of data to Google Drive. """
    save_path = os.path.join(OUTPUT_DIR, filename)

    with open(save_path, 'wb') as f:
        pkl.dump(data, f)

    print(f"Progress saved at {datetime.now().strftime('%Y-%m-%d %H:%M:%S')} to {save_path}")

In [9]:
PROGRESS_FILE = os.path.join(OUTPUT_DIR, "processed_files.log")

def get_processed_files():
    if os.path.exists(PROGRESS_FILE):
        with open(PROGRESS_FILE, 'r') as f:
            return set(f.read().splitlines())
    return set()

def update_progress(filename):
    with open(PROGRESS_FILE, 'a') as f:
        f.write(f"{filename}\n")

In [26]:
# ========== Processing Pipeline ==========
import json
import pickle as pkl
from langchain.docstore.document import Document
import time

# Save progress every 15 minutes
interval_minutes = 15

def process_files():
    extract_files()

    all_docs = [] # for storing all the documents

    # Track when the last save occurred
    last_save_time = time.time()
    processed = get_processed_files()

    # Get all text files from nested directory
    base_dir = os.path.join(EXTRACT_DIR, "elmundo_chunked_es_page1_40years")
    txt_files = [f for f in os.listdir(base_dir) if f.endswith('.txt')]

    for filename in tqdm(txt_files, desc="Processing files"):

        if filename in processed:
            # Skip already processed files
            continue

        input_path = os.path.join(base_dir, filename)
        output_path = os.path.join(OUTPUT_DIR, f"cleaned_{filename}")

        with open(input_path, 'r', encoding='utf-8', errors='ignore') as f: # open current text file
            raw_text = f.read()

        try:
            # gets gpt-4o-mini JSON object with 'metadata' and 'text' fields:
            json_object = json.loads(correct_with_openai(raw_text, filename))  # OpenAI version

            cleaned_text = json_object['text']  # get the text from the gpt-4o-mini model

            with open(output_path, 'w', encoding='utf-8') as f: # save text on google drive
                f.write(cleaned_text)

            print(f"Processed: {filename} -> Saved to Drive")

            doc = Document(                           # convert text to a langchain text object (for use on Chroma later)
                page_content=json_object['text'],
                metadata=json_object['metadata']
            )
            all_docs.append(doc)                      # append docs to list

            # Update the processed log
            update_progress(filename)

        except Exception as e:
            print(f"Error processing {filename}: {str(e)}")
            continue

        current_time = time.time()
        if (current_time - last_save_time) >= (interval_minutes * 60):
            save_progress(all_docs)
            last_save_time = current_time  # Update the last save time

    # Save all_docs as pkl file
    with open(os.path.join(OUTPUT_DIR, "all_docs.pkl"), 'wb') as f:
        pkl.dump(all_docs, f)

    with open("all_docs.pkl", 'wb') as f:
        pkl.dump(all_docs, f)

    return all_docs

In [27]:
all_docs = process_files()

**************************************************
Extracted files to: /tmp/extracted


Processing files:   0%|          | 1/1668 [00:06<2:55:24,  6.31s/it]

Processed: 19220527_1.txt -> Saved to Drive


Processing files:   0%|          | 2/1668 [00:12<2:59:48,  6.48s/it]

Processed: 19470118_1.txt -> Saved to Drive


...

Processed: 19280303_1.txt -> Saved to Drive


Processing files: 100%|█████████▉| 1667/1668 [3:09:45<00:05,  5.19s/it]

Processed: 19360208_1.txt -> Saved to Drive


Processing files: 100%|██████████| 1668/1668 [3:10:02<00:00,  6.84s/it]

Processed: 19460209_1.txt -> Saved to Drive


In [ ]:
## ========== Chroma ==========
from langchain_huggingface.embeddings import HuggingFaceEmbeddings

sentence_transformer_embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
print("Initialized SentenceTransformer embeddings.")

# Load all documents into Chroma
db = Chroma.from_documents(all_docs, sentence_transformer_embeddings, persist_directory="./chroma_db_clean_huggingface")
print('All documents loaded and embedded.(huggingface)')

In [ ]:
## trying to import the thing in my head

In [35]:
all_docs

[Document(metadata={'filename': '19220527_1.txt', 'date': 'May 27, 1922', 'locations': ['San Juan', 'Caguas', 'Arecibo']}, page_content="In the office of the Free Federation, we found Senator and socialist leader Santiago Iglesias, with whom we discussed various economic and political issues. He praised the Worker Indemnity Commission, referring to it as one of the laws with the most humanitarian spirit. Iglesias mentioned that critics claimed 86% of the Commission's income goes to salaries, while only 14% is for worker indemnities; he deemed this assertion exaggerated and called for a clear report on the Commission's finances. He also expressed concern that shipping companies are not contributing to the Commission, arguing that since Puerto Rico is not incorporated into the United States, local laws should apply. Iglesias urged lawyers to address this matter in court so that dock workers receive the benefits of the law. We hope to interview the Commission's president for his perspecti